<a href="https://colab.research.google.com/github/sascucho/Rendimiento-promedio-de-produccion-estado-de-Para-1990-2023/blob/main/An%C3%A1lisis_Par%C3%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Specify the file path
file_path = "/content/drive/MyDrive/CIENCIA DE DATOS/CORTE 2/PROYECTO/Datos Pará.xlsx"

# Load the data from the specified range, skipping the first column as data
df = pd.read_excel(file_path, sheet_name='Tabela', skiprows=3, nrows=35, usecols="B:BU", header=None)

# Set the first column as the index (Years)
df = df.set_index(df.iloc[:, 0])

# Drop the duplicate index column
df = df.drop(df.columns[0], axis=1)

display(df.head())

# Store the first row as the header
header = df.iloc[0]

# Drop the first row from the DataFrame to work with data only
df_data = df.iloc[1:].copy()

# Replace specified characters with '0' in the data part, explicitly handling downcasting
df_data.replace(['...', '..', '-'], '0', inplace=True)
df_data = df_data.infer_objects(copy=False)

# Convert the data part to numeric, coercing errors to NaN
df_data = df_data.apply(pd.to_numeric, errors='coerce')

# Identify columns with only NaN values after conversion (these were the non-numeric columns)
non_numeric_cols_after_conversion = df_data.columns[df_data.isnull().all()]

# Drop non-numeric columns from the data part
df_cleaned = df_data.drop(columns=non_numeric_cols_after_conversion)

# Fill any remaining NaN values (from coercion) with 0
df_cleaned = df_cleaned.fillna(0)

# Identify columns where the sum of values is 0
cols_to_drop_zeros = df_cleaned.columns[(df_cleaned.sum() == 0)]

# Drop the identified columns with all zeros
df_cleaned_dropped_zeros = df_cleaned.drop(columns=cols_to_drop_zeros)

# Set the column names of the final cleaned data using the values from the header
df_cleaned_dropped_zeros.columns = header[df_cleaned_dropped_zeros.columns].values

df_cleaned_dropped_zeros.index.name = 'Año'
print(df_cleaned_dropped_zeros.shape)
display(df_cleaned_dropped_zeros.head())

# Get the data for the last year
last_year_data = df_cleaned_dropped_zeros.iloc[-1]

# Sort the crops by their value in the last year in descending order
sorted_crops_last_year = last_year_data.sort_values(ascending=False)

# Convert the sorted Series to a DataFrame for saving to Excel
sorted_crops_df = sorted_crops_last_year.to_frame(name='Rendimiento Medio')

# Specify the output file path in Google Drive
output_file_path = "/content/drive/MyDrive/CIENCIA DE DATOS/CORTE 2/PROYECTO/Clasificación_cultivos_último_año.xlsx"

# Save the sorted data to a new Excel file
sorted_crops_df.to_excel(output_file_path)

print(f"La clasificación de cultivos del último año ha sido guardada en: {output_file_path}")
display(sorted_crops_df.head())

# Calculate the standard deviation for each crop
variability_by_crop = df_cleaned_dropped_zeros.std()

# Convert the result to a DataFrame
variability_df = variability_by_crop.to_frame(name='Desviación Estándar del Rendimiento')

# Sort the crops by standard deviation in descending order to see the most variable ones first
variability_df = variability_df.sort_values(by='Desviación Estándar del Rendimiento', ascending=False)

# Specify the path to the existing Excel file
output_file_path = "/content/drive/MyDrive/CIENCIA DE DATOS/CORTE 2/PROYECTO/Clasificación_cultivos_último_año.xlsx"

# Use ExcelWriter to add a new sheet to the existing file
with pd.ExcelWriter(output_file_path, engine='openpyxl', mode='a') as writer:
    variability_df.to_excel(writer, sheet_name='Variabilidad de los cultivos')

print(f"La variabilidad de los cultivos ha sido añadida a la hoja 'Variabilidad de los cultivos' en el archivo: {output_file_path}")
display(variability_df.head())

import altair as alt

# 1. Prepare data for visualization (from df_cleaned_dropped_zeros)
# Assuming df_cleaned_dropped_zeros is already available from previous steps
df_long = df_cleaned_dropped_zeros.reset_index()

df_long = df_long.melt(
    id_vars='Año',
    var_name='Cultivo',
    value_name='Rendimiento Medio'
)

# 2. Create the selection menu
crop_names = df_long['Cultivo'].unique().tolist()
crop_selector = alt.binding_select(options=crop_names, name='Selecciona un cultivo')

# Create an interactive selection object
selection = alt.selection_point(fields=['Cultivo'], bind=crop_selector, name='selection')


# 3. Create the interactive line chart
base = alt.Chart(df_long).mark_line().encode(
    x=alt.X('Año:O', axis=alt.Axis(title='Año')),
    y=alt.Y('Rendimiento Medio:Q', axis=alt.Axis(title='Rendimiento Medio')),
    color='Cultivo:N',
    tooltip=['Año', 'Cultivo', 'Rendimiento Medio']
).add_params(
    selection
).transform_filter(
    selection
)

line_chart = base

# 4. Configure and display the chart
line_chart = line_chart.properties(
    title='Evolución del Rendimiento Medio de Cultivos a lo largo del Tiempo'
)

display(line_chart)

# Calculate the sum of all crops for each year
df_cleaned_dropped_zeros['Total_Valor'] = df_cleaned_dropped_zeros.sum(axis=1)

# Melt the dataframe to long format for Altair
melted_datos = df_cleaned_dropped_zeros.reset_index().melt(id_vars=['Año', 'Total_Valor'], var_name='Cultivo', value_name='Rendimiento')

# Calculate the average yield for each crop to determine the top 10
# Exclude the 'Total_Valor' column from the mean calculation
average_yield = df_cleaned_dropped_zeros.drop(columns=['Total_Valor']).mean()
top_10_crops = average_yield.sort_values(ascending=False).head(10).index.tolist()

# Filter the melted dataframe to include only the top 10 crops
melted_top_10 = melted_datos[melted_datos['Cultivo'].isin(top_10_crops)]

# Create the interactive bar chart
chart = alt.Chart(melted_top_10).mark_bar().encode(
    x=alt.X('Año:O', axis=alt.Axis(title='Año')),
    y=alt.Y('Rendimiento:Q', axis=alt.Axis(title='Rendimiento')),
    color='Cultivo:N',
    tooltip=['Año', 'Cultivo', 'Rendimiento']
).properties(
    title='Top 10 Cultivos por Rendimiento a lo largo del Tiempo'
).interactive()

display(chart)

,2,3,4,5,6,7,8,9,10,11,...,63,64,65,66,67,68,69,70,71,72
1,,,,,,,,,,,,,,,,,,,,,
NaN,Abacate,Abacaxi*,Açaí,Alfafa fenada,Algodão arbóreo (em caroço),Algodão herbáceo (em caroço),Alho,Amendoim (em casca),Arroz (em casca),Aveia (em grão),...,Sisal ou agave (fibra),Soja (em grão),Sorgo (em grão),Tangerina,Tomate,Trigo (em grão),Triticale (em grão),Tungue (fruto seco),Urucum (semente),Uva
1990.0,54111,19300,...,...,-,550,-,444,1162,-,...,-,-,-,268763,23880,-,...,-,1098,-
1991.0,45462,19310,...,...,-,587,-,437,1348,-,...,-,-,-,271563,24469,-,...,-,1269,-
1992.0,45083,19473,...,...,-,564,-,411,1262,-,...,-,-,-,273627,25035,-,...,-,1187,-
1993.0,45140,18895,...,...,-,522,-,1735,1475,-,...,-,-,-,241619,25013,-,...,-,996,-


(34, 40)


/tmp/ipython-input-2665833669.py:24: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_data.replace(['...', '..', '-'], '0', inplace=True)


,Abacate,Abacaxi*,Açaí,Algodão herbáceo (em caroço),Amendoim (em casca),Arroz (em casca),Banana (cacho),Batata-doce,Borracha (látex coagulado),Cacau (em amêndoa),...,Melão,Milho (em grão),Palmito,Pimenta-do-reino,Soja (em grão),Sorgo (em grão),Tangerina,Tomate,Urucum (semente),Uva
Año,,,,,,,,,,,,,,,,,,,,,
1990.0,54111,19300,0,550,444,1162,1589,6141,789,593,...,4078,1187,0,2308,0,0,268763,23880,1098,0
1991.0,45462,19310,0,587,437,1348,1462,5410,643,544,...,4333,1265,0,2267,0,0,271563,24469,1269,0
1992.0,45083,19473,0,564,411,1262,1395,5000,555,572,...,3857,1188,0,991,0,0,273627,25035,1187,0
1993.0,45140,18895,0,522,1735,1475,1507,4407,494,597,...,6833,1310,0,1731,0,0,241619,25013,996,0
1994.0,31916,21349,0,414,1843,1435,1564,7888,533,637,...,7555,1273,0,1555,0,0,175016,27271,1037,0


La clasificación de cultivos del último año ha sido guardada en: /content/drive/MyDrive/CIENCIA DE DATOS/CORTE 2/PROYECTO/Clasificación_cultivos_último_año.xlsx


,Rendimiento Medio
Cana-de-açúcar,69194
Abacate,32222
Abacaxi*,24475
Tomate,23975
Melancia,21125


La variabilidad de los cultivos ha sido añadida a la hoja 'Variabilidad de los cultivos' en el archivo: /content/drive/MyDrive/CIENCIA DE DATOS/CORTE 2/PROYECTO/Clasificación_cultivos_último_año.xlsx


,Desviación Estándar del Rendimiento
Tangerina,90713.159732
Limão,85148.524738
Maracujá,47271.507113
Laranja,41871.352163
Manga,32254.007963


alt.Chart(...)

alt.Chart(...)